<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Evolution CA"

**Tags:** #fec #finance #snippet #operation #ca #revenue

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser l'évolution du CA entre l'année N et l'année N-1.

## Input

### Import libraries

In [1]:
import naas_data_product

✅ utils file '/home/ftp/FEC-engine/utils/datalake.ipynb' successfully loaded.


### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_evolution_ca"

## Model

### Récupération du dernier fichier input

In [3]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/bdd_fin/20230601154259_FEC_bdd_fin.csv
✅ Row fetched: 4532


,NOM_FICHIER,ENTITY,PERIOD,DATE,MONTH,MONTH_INDEX,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT,VALUE
0,000000000FEC20181231.txt,000000000,201812,2018-01-01,Jan,1,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.6,0.0,4006.6


### Création du dataset "EVOLUTION_CA"

In [4]:
# Creation du dataset evol_ca
dataset_evol_ca = df_input.copy()

# Filtre COMPTE_NUM = Chiffre d'Affaire (RUBRIQUE N1)
dataset_evol_ca = dataset_evol_ca[
    dataset_evol_ca["COMPTE_NUM"].str.contains(r"^70|^71|^72")
]

# Regroupement
to_group = ["ENTITY", "PERIOD", "MONTH", "MONTH_INDEX", "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
dataset_evol_ca = dataset_evol_ca.groupby(to_group, as_index=False).agg(to_agg)

dataset_evol_ca["VALUE"] = dataset_evol_ca["VALUE"].abs()


# Calcul de la somme cumulée
dataset_evol_ca = dataset_evol_ca.sort_values(
    by=["ENTITY", "PERIOD", "MONTH_INDEX"]
).reset_index(drop=True)
dataset_evol_ca["MONTH_INDEX"] = pd.to_datetime(
    dataset_evol_ca["MONTH_INDEX"], format="%m"
).dt.strftime("%m")
dataset_evol_ca["VALUE_CUM"] = dataset_evol_ca.groupby(
    ["ENTITY", "PERIOD"], as_index=True
).agg({"VALUE": "cumsum"})


dataset_evol_ca["VALUE_D"] = (dataset_evol_ca["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_evol_ca["VALUE_CUM_D"] = (dataset_evol_ca["VALUE_CUM"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")

# Affichage du modèle de donnée
dataset_evol_ca

,ENTITY,PERIOD,MONTH,MONTH_INDEX,RUBRIQUE_N3,VALUE,VALUE_CUM,VALUE_D,VALUE_CUM_D
0,000000000,201712,Jan,01,PRODUITS FINIS EXONÉRÉS,42550.15,42550.15,42.6 k€,42.6 k€
1,000000000,201712,Feb,02,PRODUITS FINIS EXONÉRÉS,40916.60,83466.75,40.9 k€,83.5 k€
2,000000000,201712,Mar,03,PRODUITS FINIS EXONÉRÉS,64832.82,148299.57,64.8 k€,148.3 k€
3,000000000,201712,Apr,04,PRODUITS FINIS EXONÉRÉS,48396.90,196696.47,48.4 k€,196.7 k€
4,000000000,201712,May,05,PRODUITS FINIS EXONÉRÉS,44719.45,241415.92,44.7 k€,241.4 k€
5,000000000,201712,Jun,06,PRODUITS FINIS EXONÉRÉS,59904.10,301320.02,59.9 k€,301.3 k€
6,000000000,201712,Jul,07,PRODUITS FINIS EXONÉRÉS,49540.65,350860.67,49.5 k€,350.9 k€
7,000000000,201712,Sep,09,PRODUITS FINIS EXONÉRÉS,48302.80,399163.47,48.3 k€,399.2 k€
8,000000000,201712,Oct,10,PRODUITS FINIS EXONÉRÉS,55081.15,454244.62,55.1 k€,454.2 k€
9,000000000,201712,Nov,11,PRODUITS FINIS EXONÉRÉS,44492.10,498736.72,44.5 k€,498.7 k€


## Output

### Sauvegarde des fichiers en csv

In [5]:
save_df(dataset_evol_ca, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/dataset_evolution_ca/20230601154912_FEC_dataset_evolution_ca.csv
